# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [2]:
#!pip install -qU ragas==0.2.10

In [3]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jsmejia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jsmejia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [22]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [23]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ffa116'. Skipping!
Property 'summary' already exists in node 'a12c39'. Skipping!
Property 'summary' already exists in node '0f82c1'. Skipping!
Property 'summary' already exists in node '4edf22'. Skipping!
Property 'summary' already exists in node 'c1e5c0'. Skipping!
Property 'summary' already exists in node 'ecf6c5'. Skipping!
Property 'summary' already exists in node '12ee78'. Skipping!
Property 'summary' already exists in node 'bbf039'. Skipping!
Property 'summary' already exists in node 'c4d622'. Skipping!
Property 'summary' already exists in node '21c113'. Skipping!
Property 'summary' already exists in node 'b95636'. Skipping!
Property 'summary' already exists in node 'd8196f'. Skipping!
Property 'summary' already exists in node 'd34953'. Skipping!
Property 'summary' already exists in node 'fca07c'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '0f82c1'. Skipping!
Property 'summary_embedding' already exists in node '12ee78'. Skipping!
Property 'summary_embedding' already exists in node 'a12c39'. Skipping!
Property 'summary_embedding' already exists in node 'bbf039'. Skipping!
Property 'summary_embedding' already exists in node 'c4d622'. Skipping!
Property 'summary_embedding' already exists in node '4edf22'. Skipping!
Property 'summary_embedding' already exists in node 'ffa116'. Skipping!
Property 'summary_embedding' already exists in node 'd8196f'. Skipping!
Property 'summary_embedding' already exists in node 'fca07c'. Skipping!
Property 'summary_embedding' already exists in node 'c1e5c0'. Skipping!
Property 'summary_embedding' already exists in node 'ecf6c5'. Skipping!
Property 'summary_embedding' already exists in node 'b95636'. Skipping!
Property 'summary_embedding' already exists in node 'd34953'. Skipping!
Property 'summary_embedding' already exists in node '21c113'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answer:

SingleHopSpecificQuerySynthesizer: Generates simple question–answer pairs (fact-based) based on a single piece of data. These are very specific types of questions.

MultiHopAbstractQuerySynthesizer: Generates questions that borrow from multiple references in the original context. The questions are less specific, focusing on higher-level concepts or implications rather than direct facts.

MultiHopSpecificQuerySynthesizer: Generates questions that borrow from multiple references in the original context, but at a more specific or detailed level (involving multiple detailed facts).




Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of Title IV in relati...,[Direct Loan Eligibility After an Enrollment S...,"Title IV program funds, including Direct Loans...",single_hop_specifc_query_synthesizer
1,What does 34 CFR 685.303 specify regarding dir...,[Direct Loan Disbursements to Students Who Tem...,34 CFR 685.303(b)(3)(iv) addresses direct loan...,single_hop_specifc_query_synthesizer
2,What counseling requirements must student Dire...,[Chapter 2 Direct Loan Counseling Counseling O...,All student Direct Loan borrowers must complet...,single_hop_specifc_query_synthesizer
3,What is FFEL and how does it relate to loans l...,[Entrance Counseling Entrance counseling is re...,Federal Family Education Loan (FFEL) refers to...,single_hop_specifc_query_synthesizer
4,What is the NSLDS?,[provided by those companies are already offer...,"The context does not explicitly define NSLDS, ...",single_hop_specifc_query_synthesizer
5,what federal student assistance programs and e...,[<1-hop>\n\nprovided by those companies are al...,The context explains that federal student assi...,multi_hop_abstract_query_synthesizer
6,How do late disbursements of Direct Loans rela...,[<1-hop>\n\nDirect Loan Eligibility After an E...,"According to the guidance, late disbursements ...",multi_hop_abstract_query_synthesizer
7,How does a temporary cease of half-time enroll...,[<1-hop>\n\nDirect Loan Eligibility After an E...,A student who temporarily ceases to be enrolle...,multi_hop_abstract_query_synthesizer
8,Wha is the requirment for exit counseling acco...,[<1-hop>\n\nRequired Content for Exit Counseli...,"According to 34 CFR 685.304(b)(4), the require...",multi_hop_specific_query_synthesizer
9,How do the requirements for Direct Loan disbur...,[<1-hop>\n\nDirect Loan Eligibility After an E...,The requirements in Chapter 2 specify that fir...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/30 [00:00<?, ?it/s]

Property 'summary' already exists in node 'f0b061'. Skipping!
Property 'summary' already exists in node '952be1'. Skipping!
Property 'summary' already exists in node 'c574a3'. Skipping!
Property 'summary' already exists in node 'bab03a'. Skipping!
Property 'summary' already exists in node 'af42e1'. Skipping!
Property 'summary' already exists in node '8a6554'. Skipping!
Property 'summary' already exists in node '1f41a9'. Skipping!
Property 'summary' already exists in node 'be8d43'. Skipping!
Property 'summary' already exists in node '3581a8'. Skipping!
Property 'summary' already exists in node 'df3d5a'. Skipping!
Property 'summary' already exists in node '7707fc'. Skipping!
Property 'summary' already exists in node '2d4660'. Skipping!
Property 'summary' already exists in node '624519'. Skipping!


...

Property 'summary_embedding' already exists in node 'f0b061'. Skipping!
Property 'summary_embedding' already exists in node '952be1'. Skipping!
Property 'summary_embedding' already exists in node '624519'. Skipping!
Property 'summary_embedding' already exists in node '1f41a9'. Skipping!
Property 'summary_embedding' already exists in node 'c574a3'. Skipping!
Property 'summary_embedding' already exists in node 'af42e1'. Skipping!
Property 'summary_embedding' already exists in node '7707fc'. Skipping!
Property 'summary_embedding' already exists in node '2d4660'. Skipping!
Property 'summary_embedding' already exists in node 'df3d5a'. Skipping!
Property 'summary_embedding' already exists in node 'bab03a'. Skipping!
Property 'summary_embedding' already exists in node 'be8d43'. Skipping!
Property 'summary_embedding' already exists in node '8a6554'. Skipping!
Property 'summary_embedding' already exists in node '3581a8'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [37]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What information is available regarding Chapte...,"[information, see the discussion under <Direct...",The discussion under Chapter 3 covers details ...,single_hop_specifc_query_synthesizer
1,Who can borow Direct PLUS Loan for a dependnt ...,[Definition of <Parent= for Direct PLUS Loan P...,Individuals who can borrow a Direct PLUS Loan ...,single_hop_specifc_query_synthesizer
2,Chapter 3 what about direct loan and chapter 3?,[Direct Loan Eligibility After an Enrollment S...,The context discusses Direct Loan eligibility ...,single_hop_specifc_query_synthesizer
3,What does 34 CFR 685.303(b)(3)(iv) say about d...,[Direct Loan Disbursements to Students Who Tem...,34 CFR 685.303(b)(3)(iv) relates to direct loa...,single_hop_specifc_query_synthesizer
4,what info for loan help and exit counseling do...,[<1-hop>\n\nprovided by those companies are al...,Students need to provide current personal info...,multi_hop_abstract_query_synthesizer
5,Whay impact does enrolement ceasation hav on D...,[<1-hop>\n\nDirect Loan Eligibility After an E...,The context explains that if a student ceases ...,multi_hop_abstract_query_synthesizer
6,Wht are the loan terms and condtions for stude...,[<1-hop>\n\nEntrance Counseling Entrance couns...,Entrance counseling is required for all first-...,multi_hop_abstract_query_synthesizer
7,Eligibility criteria for Direct PLUS Loans tes...,"[<1-hop>\n\ninformation, see the discussion un...",The context explains that Direct PLUS Loans ar...,multi_hop_abstract_query_synthesizer
8,What are the requierd content for exit counsel...,[<1-hop>\n\nRequired Content for Exit Counseli...,"According to 34 CFR 685.304(b)(4), the require...",multi_hop_specific_query_synthesizer
9,Can you explain how Chapter 3 details the elig...,[<1-hop>\n\nDirect Loan Eligibility After an E...,Chapter 3 explains that if a student who recei...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [25]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data 2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data 2"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [26]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [27]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [30]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [32]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [35]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct PLUS Loan or student Federal PLUS Loan  \n- Subsidized Federal Stafford Loans  \n- Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n- Direct Subsidized Loan  \n- Direct Unsubsidized Loan  \n- Student Direct PLUS Loan  \n- Direct Consolidation Loan  \n- Federal Consolidation Loan\n\nAdditionally, it is noted that Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010, and no new FFEL loans are being made after that date. Direct Subsidized, Unsubsidized, and PLUS Loans are current loan types under the Direct Loan Program.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [36]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [38]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"], # type: ignore
        "reference": example.outputs["answer"], # type: ignore
        "input": example.inputs["question"], # type: ignore
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

The "qa evaluator" (question–answer evaluator) compares the predicted (output) answer against the expected (reference) answer using a reference-based method. It aims to answer the question: Does the output answer contain the correct information based on the reference answer?
In this context, we are measuring correctness based on semantic matches between the output and the reference. The evaluator_type in this example is labeled_criteria, since we have a reference answer (label).

The "labeled_helpfulness_evaluator" measures a specific criterion, in this case "helpfulness", defined as:
"Is this submission helpful to the user, taking into account the correct reference answer?"
To that end, and as specified in the criteria, it also uses a reference answer. This is a slightly more subjective metric.

Finally, we have the "empathy_evaluator", which also measures a specific criterion, in this case "empathy":
"Is this response empathetic? Does it make the user feel like they are being heard?"
This criterion does not consider a reference answer, making it even more subjective than the previous two. The subjective nature becomes especially clear when evaluating qualities such as style and tone.

## LangSmith Evaluation

In [39]:
evaluate(
    rag_chain.invoke, # type: ignore
    data=dataset_name,
    evaluators=[
        qa_evaluator, # type: ignore
        labeled_helpfulness_evaluator, # type: ignore
        empathy_evaluator # type: ignore
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'timely-name-22' at:
https://smith.langchain.com/o/8127b92a-2915-4164-8774-631e7b7a15e2/datasets/90e79cf0-741a-4bf4-9c93-80fc31b0ce08/compare?selectedSessions=4e4a118a-99b2-4237-9011-59f64aeadd6d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,how do direct unsubsidized loans work if paren...,Based on the provided context:\n\nIf the paren...,None,if parents refuse to fill out fafsa or stop fi...,0,0,0,7.004163,7b1431c5-3866-4c03-a338-83c20c6ab810,6e9f8260-a70b-4e44-9153-ffa477c0503c
1,Whas is a Direct PLUS Loan and whay is it impo...,A Direct PLUS Loan is a federal student loan a...,None,A Direct PLUS Loan is a type of federal loan t...,1,1,0,2.803628,fe26481d-a308-4e38-90ac-c0aac5508dd0,b0e6753f-b506-41cd-8157-5ca7a536e211
2,Can you explain how Chapter 3 details the elig...,"Based on the provided context, Chapter 3 inclu...",None,Chapter 3 explains that if a student who recei...,1,0,0,8.485634,ef50f1b3-a620-4deb-b402-6d4cee785c9a,8fa4c398-e7ed-4fe7-9718-c86a12fc9919
3,What are the requierd content for exit counsel...,Based on the provided context:\n\n- The requir...,None,"According to 34 CFR 685.304(b)(4), the require...",1,0,0,8.536196,7674f3df-a773-49a7-8c27-0ed77f8088ee,ee7e05e5-51be-4d09-9917-e71c78ff2f98
4,Eligibility criteria for Direct PLUS Loans tes...,"Based on the provided context, the eligibility...",None,The context explains that Direct PLUS Loans ar...,1,0,0,10.158068,0f71e6b6-d5e1-434d-9f71-d09ee643e2cc,97496450-e9f6-49d0-93bf-2b9b1f642bad
5,Wht are the loan terms and condtions for stude...,"Based on the provided context, the loan terms ...",None,Entrance counseling is required for all first-...,1,1,0,3.865106,f0375b4e-fc7b-4f1d-be17-d748a9d53018,8a2c2873-8178-4274-b5f1-1f308904c006
6,Whay impact does enrolement ceasation hav on D...,"Based on the provided context, if a student wh...",None,The context explains that if a student ceases ...,1,0,0,5.060083,62ac10b7-54ca-4d01-b47c-cc20c6a766b0,ac007c80-683e-4555-a94d-b7b4af9d8c35
7,what info for loan help and exit counseling do...,"Based on the provided context, students who ha...",None,Students need to provide current personal info...,1,1,0,7.336361,f63da2bb-7ea2-468a-8965-4ed28361a40b,d5b03462-69c1-477d-a008-a28df82da171
8,What does 34 CFR 685.303(b)(3)(iv) say about d...,I don't know.,None,34 CFR 685.303(b)(3)(iv) relates to direct loa...,0,0,0,0.897156,393afaf9-8394-4ba8-9297-9e0fe5f55f04,b21bf51d-7b60-4e4e-82d1-fbeae4a15f44
9,Chapter 3 what about direct loan and chapter 3?,Chapter 3 of the document covers Direct Loan O...,None,The context discusses Direct Loan eligibility ...,0,0,0,3.800473,8e0952e7-8207-4c3a-be9e-9033f53ab2f9,37f473c5-0ef3-4383-b4d6-cb6c8f08bee5


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [40]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [41]:
rag_documents = docs

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

Increasing the chunk size means each chunk contains more context, which can:

(a) Improve retrieval quality, since there's a higher chance that relevant information is included in a single chunk.

(b) Reduce fragmentation, helping the retriever return a more complete answer context.

However, if we go too far, we can also dilute the specific content of the chunk in the embedding, resulting in diminishing returns.

In [43]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

Using a larger embedding model typically provides a richer embedding space, which means:

(a) The embeddings are more descriptive and better capture semantic meaning, nuances, and context from the input text, offering "finer granularity."

(b) Potentially improved retrieval performance, as similarity search becomes more aligned with the actual intent and content of the query, reflecting "increased sensitivity."

However, this comes with trade-offs: increased cost and latency due to the larger model size, as well as the possibility of overfitting or oversensitivity.

In [44]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [45]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [46]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [47]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information provided, there are a few types of loans available to students and their parents to help cover college expenses:\n\n1. **Direct Subsidized Loans** – These loans are based on the student’s financial need, and the government pays the interest while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These loans are available regardless of financial need, and interest accrues while the student is in school.\n\n3. **Direct PLUS Loans** – These can be taken out by the parents of dependent students to cover the cost of attendance, assuming they meet eligibility requirements. There’s no fixed loan limit for PLUS loans, but they cannot exceed the student's total cost of attendance minus other aid.\n\nIf a dependent student's parent is ineligible for a Direct PLUS Loan, the student may qualify for additional Direct Unsubsidized Loans to help cover costs.\n\nI hope this helps clarify the kinds of loans availabl

Finally, we can evaluate the new chain on the same test set!

In [48]:
evaluate(
    empathy_rag_chain.invoke, # type: ignore
    data=dataset_name,
    evaluators=[
        qa_evaluator, # type: ignore
        labeled_helpfulness_evaluator, # type: ignore
        empathy_evaluator # type: ignore
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'aching-event-14' at:
https://smith.langchain.com/o/8127b92a-2915-4164-8774-631e7b7a15e2/datasets/90e79cf0-741a-4bf4-9c93-80fc31b0ce08/compare?selectedSessions=a5600b26-6584-448f-a55b-d95397567454




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,how do direct unsubsidized loans work if paren...,Thank you for your thoughtful question. Based ...,None,if parents refuse to fill out fafsa or stop fi...,1,1,1,6.107908,7b1431c5-3866-4c03-a338-83c20c6ab810,96371950-b4d2-480e-8ee0-31c37e678fe7
1,Whas is a Direct PLUS Loan and whay is it impo...,Thank you for your thoughtful question. Based ...,None,A Direct PLUS Loan is a type of federal loan t...,1,1,1,4.544104,fe26481d-a308-4e38-90ac-c0aac5508dd0,889f9f03-d93a-4103-989e-64cdd6c718f1
2,Can you explain how Chapter 3 details the elig...,Thank you for your thoughtful question. I can ...,None,Chapter 3 explains that if a student who recei...,1,0,1,6.670454,ef50f1b3-a620-4deb-b402-6d4cee785c9a,efc1d797-6569-45de-830c-6caf9ad836db
3,What are the requierd content for exit counsel...,Thank you for your thoughtful question. Based ...,None,"According to 34 CFR 685.304(b)(4), the require...",0,0,1,6.367842,7674f3df-a773-49a7-8c27-0ed77f8088ee,1a5bd525-1457-41c1-877b-3ba7abb831e5
4,Eligibility criteria for Direct PLUS Loans tes...,Thank you for your question—navigating loan el...,None,The context explains that Direct PLUS Loans ar...,1,0,1,14.943816,0f71e6b6-d5e1-434d-9f71-d09ee643e2cc,f95b718d-1f4a-401c-9936-e0f753ca4e3a
5,Wht are the loan terms and condtions for stude...,Thank you for your question about the loan ter...,None,Entrance counseling is required for all first-...,1,0,1,8.681634,f0375b4e-fc7b-4f1d-be17-d748a9d53018,2606d2c4-4859-434d-b03e-a1da2cb5ab5a
6,Whay impact does enrolement ceasation hav on D...,Thank you for your question. I understand that...,None,The context explains that if a student ceases ...,1,1,1,4.336006,62ac10b7-54ca-4d01-b47c-cc20c6a766b0,dec9398d-9a69-49c8-8a87-a051c044a558
7,what info for loan help and exit counseling do...,Thank you for your thoughtful question. Based ...,None,Students need to provide current personal info...,1,1,1,7.720474,f63da2bb-7ea2-468a-8965-4ed28361a40b,b4c46b3f-00e0-4ade-982c-90c644dedae7
8,What does 34 CFR 685.303(b)(3)(iv) say about d...,Thank you for your thoughtful question. Based ...,None,34 CFR 685.303(b)(3)(iv) relates to direct loa...,1,1,1,4.694991,393afaf9-8394-4ba8-9297-9e0fe5f55f04,b4b4532a-77dd-4474-90de-d890bad2374a
9,Chapter 3 what about direct loan and chapter 3?,Thank you for your question! Based on the cont...,None,The context discusses Direct Loan eligibility ...,0,0,1,5.084227,8e0952e7-8207-4c3a-be9e-9033f53ab2f9,bbceca3f-e400-4117-852c-d3ca39694efd


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![alt text](comparison_results.jpg "optional title")

##### ✅ Answer:

From the comparison table, we can clearly see that results are improving across all areas.

Empathy is the most striking example, increasing from 0 in the baseline RAG to 1 in the empathy RAG.
This improvement can be directly attributed to the explicit instruction in the generation prompt of the empathy RAG chain:
"You must answer the question using empathy and kindness, and make sure the user feels heard."

For the other two metrics, correctness and helpfulness, the performance gain is similar, with an approximate delta of 0.16.
In the case of correctness, the improvement could be due to the increased chunk size (providing more context) and the use of a larger model (offering finer-grained encoding and higher sensitivity). With these factors, the LLM has a better chance of producing the “correct” answer.

As for helpfulness, this metric is in a similar situation to correctness, potentially influenced by chunk size and the type of embedding model used (larger model). However, the impact is less clear at the intuition level, in part due to the subjective nature of the metric itself: "Is this submission helpful to the user, taking into account the correct reference answer?"

---

**Lessons Learned and NOT Learned yet:**  

**3 Lessons Learned**

1. The pipeline for generating synthetic data to evaluate a RAG application consists of the following steps:
document loading, document chunking, chunk embedding, creation of a knowledge graph, application of transformations to the nodes in the graph (create node relations), and LLM-based generation of paired data (user input, reference contexts, and reference answer).

2. With just two simple commands in the RAGAS framework, we can generate a complete synthetic dataset for agent evaluation.
These two commands are TestsetGenerator, which creates the RAGAS generation instance, and generate_with_langchain_docs, which performs the actual generation. Together, they abstract the following steps:
document chunking, chunk embedding, creation of a knowledge graph, application of transformations to graph nodes (create node relations), and LLM-based data generation.

3. RAGAS metrics by themselves are not very informative in isolation. They become valuable when we compare the differences between a baseline and new experiments where different parameters are used in the evaluation process.
Examples of such parameters in a RAG application include using a more detailed prompt, changing the document chunking strategy (such as the chunking method or parameters), and switching to a different embedding model.
